#Telco Customer Churn Analysis

Subject: Predicting Customer Churn in Telecommunications Sector

Our project will follow a 7-step methodology that is closely aligned with well-known frameworks used in data science projects, such as CRISP-DM, Microsoft TDSP, and KDnuggets Modern Data Science Process. The 7-steps are as follows:

1. Data Integration and Cleaning
2. Exploratory Data Analysis (EDA)
3. Feature Engineering
4. Predictive Modelling
5. Model Evaluation
6. Explainability and Visualisation
7. Business Recommendations


## Environment Setput and Configuration

In [1]:
# For Google Colab: Mount Google Drive

from google.colab import drive
from pathlib import Path
import os

drive.mount('/content/drive')

# Define paths
project_path = '/content/drive/MyDrive/Colab Notebooks/Bootcamp/Telco_Churn_Project/'
#project_path = '/content/drive/MyDrive/Colab Notebooks/Bootcamp/TP2/Telco_Churn_Project/'

data_path = project_path + '01_Data/'
notebook_path = project_path + '02_Notebooks/'
output_path = project_path + '03_Outputs/'

#Import python libraries
!pip install polars[excel]

#import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("Setup has been completed")

MessageError: Error: credential propagation was unsuccessful

## Upload Data to Dataframes


In [ ]:
demographics_df = pl.read_excel(f"{data_path}Telco_customer_churn_demographics.xlsx")
status_df      = pl.read_excel(f"{data_path}Telco_customer_churn_status.xlsx")
population_df  = pl.read_excel(f"{data_path}Telco_customer_churn_population.xlsx")
location_df    = pl.read_excel(f"{data_path}Telco_customer_churn_location.xlsx")
services_df    = pl.read_excel(f"{data_path}Telco_customer_churn_services.xlsx")

print(f"  - Demographics: {demographics_df.shape}")
print(f"  - Status: {status_df.shape}")
print(f"  - Population: {population_df.shape}")
print(f"  - Location: {location_df.shape}")
print(f"  - Services: {services_df.shape}")

print(f" *** All external files were sucessfully imported ***")

#1.Data Integration and Cleaning

**1.1 - Data Knowledge: Display schema and head rows of each dataframe**

In [ ]:
#print(f"demographics_df.schema: {demographics_df.schema}")
print("demographics_df.schema & head rows")
display(demographics_df.head(5))


In [ ]:
print("status_df.schema & head rows")
display(status_df.head(5))


In [ ]:

print("population_df.schema & head rows")
display(population_df.head(5))


In [ ]:
print("location_df.schema & head rows")
display(location_df.head(5))


In [ ]:
print("services_df.schema & head rows")
display(services_df.head(5))

**1.2 - Data Consolidation: Include all data in one dataframe**


In [ ]:
print("\n1.2 Merging datasets...")

# Start with demographics as base
df = demographics_df.clone()   #

# Merge location, services, status and population data
df = df.join(location_df, on="Customer ID", how="left", suffix="_loc")
df = df.join(services_df, on="Customer ID", how="left", suffix="_svc")
df = df.join(status_df, on="Customer ID", how="left", suffix="_sts")
df = df.join(
    population_df.select(["Zip Code", "Population"]),
    on="Zip Code",
    how="left"
)

print(f"  - Merged dataset shape: {df.shape}")


In [ ]:
#print("df.schema & head rows")
display(df.head(5))

**1.3 - Data Consolidation: Remove duplicated columns**

In [ ]:
print("\n1.3 Removing duplicate columns...")

# Identify columns with suffixes
cols_to_drop = [
    col for col in df.columns
    if col.endswith("_loc") or col.endswith("_svc") or col.endswith("_sts")
]

# Drop them
df = df.drop(cols_to_drop)

print(f"  - Dropped {len(cols_to_drop)} duplicate columns")
print(f"  - Final dataset shape: {df.shape}")

#print("df.schema & head rows")
display(df.head(5))


**1.4 - Data Consolidation: Check Missing Values**

In [ ]:
print("\n1.4 Checking for missing values...")

# Count missing values per column
missing_summary = (
    pl.DataFrame({
        "Column": df.columns,
        "Missing_Count": [df[col].null_count() for col in df.columns],
    })
    .with_columns([
        (pl.col("Missing_Count") / len(df) * 100)
        .round(2)
        .alias("Missing_Percentage")
    ])
    .filter(pl.col("Missing_Count") > 0)
    .sort("Missing_Count", descending=True)
)

if missing_summary.height > 0:
    print("\nColumns with missing values:")
    print(missing_summary)
else:
    print(" --- No missing values found --- ")


A**ttention to this:** The Churn Category and Churn Reason fields are mostly empty. This will be limiting because these features could be important to the prediction model.

**1.5 - Data Consolidation: Save integrated dataset & Create data dictionary**

In [ ]:
print("\n1.5 Saving integrated dataset...")

# Save integrated dataset
df.write_csv(f"{output_path}telco_churn_integrated.csv")
print(f" Saved to: {output_path}telco_churn_integrated.csv")


In [ ]:
# Create data dictionary
print(" Creating data dictionary...")

data_dict = pl.DataFrame({
    "Variable": df.columns,
    "Type": [str(dtype) for dtype in df.dtypes],
    "Non_Null_Count": [df[col].len() - df[col].null_count() for col in df.columns],
    "Unique_Values": [df[col].n_unique() for col in df.columns],
    "Sample_Values": [
        str(df[col].drop_nulls().unique().limit(3).to_list()) for col in df.columns
    ],
})

# Save dictionary
data_dict.write_csv(f"{output_path}data_dictionary.csv")
print(f" Data dictionary saved to: {output_path}data_dictionary.csv")

#2.Exploratory Data Analysis

**2.1 - Target variable distribution**

In [ ]:
import plotly.express as px

print("\n2.1. Target Variable Distribution (Churn Label)...")

# --- Compute churn counts and percentages ---
churn_stats = (
    df.group_by("Churn Label")
      .agg(pl.len().alias("Count"))
      .with_columns(
          (pl.col("Count") / pl.col("Count").sum() * 100)
          .round(2)
          .alias("Percentage")
      )
      .sort("Churn Label")
)

print("\nChurn Distribution:")
for row in churn_stats.iter_rows(named=True):
    print(f"  - {row['Churn Label']}: {row['Count']} ({row['Percentage']:.2f}%)")

# --- Prepare data for Plotly pie chart ---
churn_counts = churn_stats.select(["Churn Label", "Count"]).to_pandas()

fig = px.pie(
    churn_counts,
    values="Count",
    names="Churn Label",
    title="Customer Churn Distribution",
    color_discrete_sequence=["#2ecc71", "#e74c3c"]
)

fig.write_html(f"{output_path}churn_distribution.html")
print(f"  ✓ Visualization saved: {output_path}churn_distribution.html")


**Attention to this:** The dataset is strongly unbalanced, and this issue must be addressed.

**2.2 - Descritive Statistics**

In [ ]:
print("\n2.2 Descriptive Statistics for Numerical Variables...")

# Select only numeric columns
numerical_cols = [
    col for col, dtype in df.schema.items()
    if dtype in (pl.Int8, pl.Int16, pl.Int32, pl.Int64,
                 pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
                 pl.Float32, pl.Float64)
]

desc_stats = df.select(numerical_cols).describe()
display(desc_stats)

# Save to CSV
desc_stats.write_csv(f"{output_path}descriptive_statistics.csv")

print(f"Descriptive Statistics saved to: {output_path}descriptive_statistics.csv")


**2.3 - Analysing Churn by demographic variables**

In [ ]:
def ensure_no_yes_order(df, id_col):
    desired_order = [id_col, "No", "Yes"]
    return df.select([c for c in desired_order if c in df.columns])

# By Gender ********************
gender_churn = (
    df.group_by(["Gender", "Churn Label"])
      .agg(pl.len().alias("Count"))
      .with_columns(
          (pl.col("Count") / pl.col("Count").sum().over("Gender") * 100)
          .round(2)
          .alias("Percentage")
      )
      .pivot(
          values="Percentage",
          index="Gender",
          columns="Churn Label"
      )
      .fill_null(0)
      .sort("Gender")
)

print("\nChurn Rate by Gender (%):")
gender_churn = ensure_no_yes_order(gender_churn, "Gender")
print(gender_churn)


In [ ]:
# By Group Age ***************
df = df.with_columns(
    pl.when(pl.col("Age") <= 30).then(pl.lit("<30"))
     .when(pl.col("Age") <= 50).then(pl.lit("30-50"))
     .when(pl.col("Age") <= 65).then(pl.lit("50-65"))
     .otherwise(pl.lit("65+"))
     .alias("Age_Group")
)

age_churn = (
    df.group_by(["Age_Group", "Churn Label"])
      .agg(pl.len().alias("Count"))
      .with_columns(
          (pl.col("Count") / pl.col("Count").sum().over("Age_Group") * 100)
          .round(2)
          .alias("Percentage")
      )
      .pivot(
          values="Percentage",
          index="Age_Group",
          columns="Churn Label"
      )
      .fill_null(0)
      .sort("Age_Group")
)

print("\nChurn Rate by Group Age (%):")
age_churn = ensure_no_yes_order(age_churn, "Age_Group")
print(age_churn)

In [ ]:
# Senior Citizen *********************
senior_churn = (
    df.group_by(["Senior Citizen", "Churn Label"])
      .agg(pl.len().alias("Count"))
      .with_columns(
          (pl.col("Count") / pl.col("Count").sum().over("Senior Citizen") * 100)
          .round(2)
          .alias("Percentage")
      )
      .pivot(
          values="Percentage",
          index="Senior Citizen",
          columns="Churn Label"
      )
      .fill_null(0)
      .sort("Senior Citizen")
)

print("\nChurn Rate by Senior Citizen (%):")
senior_churn = ensure_no_yes_order(senior_churn, "Senior Citizen")
print(senior_churn)

In [ ]:
# By Marital Status ***************
married_churn = (
    df.group_by(["Married", "Churn Label"])
      .agg(pl.len().alias("Count"))
      .with_columns(
          (pl.col("Count") / pl.col("Count").sum().over("Married") * 100)
          .round(2)
          .alias("Percentage")
      )
      .pivot(
          values="Percentage",
          index="Married",
          columns="Churn Label"
      )
      .fill_null(0)
      .sort("Married")
)

print("\nChurn Rate by Marital Status (%):")
married_churn = ensure_no_yes_order(married_churn, "Married")
print(married_churn)


**2.4 - Analysing Churn by service variables**

In [ ]:
print("\n2.4 Churn Analysis by Services...")

# --- Contract Type ---
contract_churn = (
    df.group_by(["Contract", "Churn Label"])
      .agg(pl.len().alias("Count"))
      .with_columns(
          (pl.col("Count") / pl.col("Count").sum().over("Contract") * 100)
          .round(2)
          .alias("Percentage")
      )
      .pivot(values="Percentage", index="Contract", columns="Churn Label")
      .fill_null(0)
      .sort("Contract")
)

print("\nChurn Rate by Contract Type (%):")
contract_churn  = ensure_no_yes_order(contract_churn,  "Contract")
print(contract_churn)


# --- Internet Type ---
internet_churn = (
    df.group_by(["Internet Type", "Churn Label"])
      .agg(pl.len().alias("Count"))
      .with_columns(
          (pl.col("Count") / pl.col("Count").sum().over("Internet Type") * 100)
          .round(2)
          .alias("Percentage")
      )
      .pivot(values="Percentage", index="Internet Type", columns="Churn Label")
      .fill_null(0)
      .sort("Internet Type")
)

print("\nChurn Rate by Internet Type (%):")
internet_churn  = ensure_no_yes_order(internet_churn,  "Internet Type")
print(internet_churn)


# --- Payment Method ---
payment_churn = (
    df.group_by(["Payment Method", "Churn Label"])
      .agg(pl.len().alias("Count"))
      .with_columns(
          (pl.col("Count") / pl.col("Count").sum().over("Payment Method") * 100)
          .round(2)
          .alias("Percentage")
      )
      .pivot(values="Percentage", index="Payment Method", columns="Churn Label")
      .fill_null(0)
      .sort("Payment Method")
)

print("\nChurn Rate by Payment Method (%):")
payment_churn   = ensure_no_yes_order(payment_churn,   "Payment Method")
print(payment_churn)


**2.5 - Analysing Churn by tenure**

In [ ]:
print("\n2.5. Churn Analysis by Tenure...")

# Create tenure group
df = df.with_columns(
    pl.when(pl.col("Tenure in Months") <= 12).then(pl.lit("0-12m"))
     .when(pl.col("Tenure in Months") <= 24).then(pl.lit("12-24m"))
     .when(pl.col("Tenure in Months") <= 48).then(pl.lit("24-48m"))
     .otherwise(pl.lit("48m+"))
     .alias("Tenure_Group")
)

# Crosstab equivalent
tenure_churn = (
    df.group_by(["Tenure_Group", "Churn Label"])
      .agg(pl.len().alias("Count"))
      .with_columns(
          (pl.col("Count") / pl.col("Count").sum().over("Tenure_Group") * 100)
          .round(2)
          .alias("Percentage")
      )
      .pivot(
          values="Percentage",
          index="Tenure_Group",
          columns="Churn Label"
      )
      .fill_null(0)
      .sort("Tenure_Group")
)

print("\nChurn Rate by Tenure Group (%):")
tenure_churn = ensure_no_yes_order(tenure_churn, "Tenure_Group")
print(tenure_churn)


**2.6 - Financial metrics analysis**

In [ ]:
print("\n2.6. Financial Metrics Analysis...")

financial_cols = ["Monthly Charge", "Total Revenue", "CLTV", "Total Charges"]

# Compute averages by churn label
financial_by_churn = (
    df.group_by("Churn Label")
      .agg([pl.col(c).mean().alias(c) for c in financial_cols])
      .sort("Churn Label")
)

print("\nAverage Financial Metrics by Churn:")
print(financial_by_churn)


**2.7 - Correlation analysis**

In [ ]:
# Select numerical columns for correlation
corr_cols = [
    "Age", "Tenure in Months", "Monthly Charge", "Total Revenue", "CLTV",
    "Satisfaction Score", "Churn Score", "Number of Referrals",
    "Avg Monthly GB Download", "Population"
]

# Compute Pearson's correlation matrix
corr_matrix = df.select(corr_cols).corr()

# Display correlation matrix
print("\nPearson's Correlation Matrix:")
print(corr_matrix)

# Save to CSV
corr_file = f"{output_path}Pearsons_correlation_matrix.csv"
corr_matrix.write_csv(corr_file)
print(f"\nPearson's Correlation matrix saved: {corr_file}")



**Heat Map: Visualize correlation matrix**

In [ ]:
# corr_matrix is a Polars DataFrame from: corr_matrix = df.select(corr_cols).corr()

# 1) Extract variable names from Polars'
if "column" in corr_matrix.columns:
    names = corr_matrix.get_column("column").to_list()
    vals_pl = corr_matrix.drop("column")
else:
    names = corr_cols
    vals_pl = corr_matrix

# 2) Convert values to pandas and set both axes labels
corr_pd = vals_pl.to_pandas()
corr_pd.index = names
corr_pd.columns = names

# 3) Display the heat map
plt.figure(figsize=(12, 10))
sns.heatmap(
    corr_pd.astype(float),
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    square=True,
    linewidths=1,
    cbar_kws={"shrink": 0.8},
)
plt.title("Pearson's Correlation Matrix of Numerical Variables", fontsize=16, fontweight="bold")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()

# Show and save
plt.show()
heatmap_path = f"{output_path}correlation_heatmap.png"
plt.savefig(heatmap_path, dpi=300, bbox_inches="tight")
print(f"Pearson's Correlation heatmap displayed and saved: {heatmap_path}")



**2.8 - Geographic analysis**  Not relevant because only one state. Probably using city will be better but even so, small cities with a small number of customers apears in the top ten.

In [ ]:
geo_churn = (
    df.group_by("City")
      .agg([
          pl.count("Customer ID").alias("Total_Customers"),
          (pl.col("Churn Label") == "Yes").sum().alias("Churned_Customers")
      ])
      .with_columns(
          ((pl.col("Churned_Customers") / pl.col("Total_Customers")) * 100)
          .round(2)
          .alias("Churn_Rate")
      )
      .sort("Churn_Rate", descending=True)
)

# Save to CSV
geo_churn.write_csv(f"{output_path}churn_by_city.csv")
print(f"  ✓ Geographic churn analysis saved: {output_path}churn_by_city.csv")

# Show top 10 Cities
print("\nTop 10 Cities by Churn Rate:")
print(geo_churn.head(10))
